In [4]:
import json
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
import matplotlib.pyplot as plt

# Load the configuration file
with open('/Users/andreshofmann/Desktop/Studies/Uol/7t/FP/stage_2/CNN/config_v8_cnn.json', 'r') as config_file:
    config = json.load(config_file)

# Paths
base_dir = config['paths']['base_dir']
csv_path = config['paths']['csv_path']
img_dir = config['paths']['img_dir']
model_save_path = config['paths']['model_save_path']

# Training parameters
validation_split = config['training']['validation_split']
fraction = config['training']['fraction']
batch_size = config['training']['batch_size']
epochs = config['training']['epochs']
initial_learning_rate = config['training']['initial_learning_rate']
learning_rate_decay = config['training']['learning_rate_decay']
patience = config['training']['patience']
reduce_lr_factor = config['training']['reduce_lr_factor']
reduce_lr_patience = config['training']['reduce_lr_patience']
min_lr = config['training']['min_lr']

# Data augmentation parameters
data_aug_params = config['data_augmentation']

# Model architecture parameters
input_shape = tuple(config['model_architecture']['input_shape'])
conv_layers = config['model_architecture']['conv_layers']
pool_size = tuple(config['model_architecture']['pool_size'])
dense_layers = config['model_architecture']['dense_layers']
dropout_rate = config['model_architecture']['dropout_rate']
batch_normalization = config['model_architecture']['batch_normalization']
l2_regularization = config['model_architecture']['l2_regularization']

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=data_aug_params['rescale'],
    rotation_range=data_aug_params['rotation_range'],
    width_shift_range=data_aug_params['width_shift_range'],
    height_shift_range=data_aug_params['height_shift_range'],
    shear_range=data_aug_params['shear_range'],
    zoom_range=data_aug_params['zoom_range'],
    horizontal_flip=data_aug_params['horizontal_flip'],
    vertical_flip=data_aug_params['vertical_flip'],
    brightness_range=data_aug_params['brightness_range'],
    validation_split=validation_split
)

train_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    img_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Input(shape=input_shape))
        for layer in conv_layers:
            model.add(Conv2D(
                filters=hp.Int('filters_' + str(layer['filters']), min_value=32, max_value=256, step=32),
                kernel_size=tuple(layer['kernel_size']),
                activation=layer['activation'],
                kernel_regularizer=tf.keras.regularizers.l2(l2_regularization)
            ))
            if batch_normalization:
                model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=pool_size))
        
        model.add(GlobalAveragePooling2D())
        
        for layer in dense_layers:
            model.add(Dense(units=hp.Int('units_' + str(layer['units']), min_value=128, max_value=1024, step=128), activation=layer['activation']))
            if batch_normalization:
                model.add(BatchNormalization())
        
        model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.2, max_value=0.7, step=0.1)))
        model.add(Dense(1, activation='sigmoid'))
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=initial_learning_rate)),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

# Hyperparameter tuner
tuner = RandomSearch(
    CNNHyperModel(),
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory=base_dir,
    project_name='skin_lesion_tuning'
)

# Callbacks
early_stopping = EarlyStopping(patience=patience, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=reduce_lr_factor, patience=reduce_lr_patience, min_lr=min_lr)

# Run the tuner
tuner.search(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model
model = tuner.hypermodel.build(best_hps)

# Train the model with the best hyperparameters
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Save the model
model.save(model_save_path)

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

Found 35790 images belonging to 2 classes.
Found 8946 images belonging to 2 classes.

Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
128               |128               |filters_32
224               |224               |filters_64
224               |224               |filters_128
160               |160               |filters_256
896               |896               |units_512
0.6               |0.6               |dropout_rate
0.0002242         |0.0002242         |learning_rate

Epoch 1/50
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 287s 255ms/step - accuracy: 0.5669 - loss: 1.4007 - val_accuracy: 0.6037 - val_loss: 1.0666 - learning_rate: 2.2420e-04
Epoch 2/50
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 283s 252ms/step - accuracy: 0.5882 - loss: 1.0843 - val_accuracy: 0.6394 - val_loss: 0.9103 - learning_rate: 2.2420e-04
Epoch 3/50
1119/1119 ━━━━━━━━━━━━━━━━━━━━ 269s 241ms/step - accuracy: 0.6028 - loss: 0.9140 - val_accuracy: 0.6527 - val_loss: 0.7665 - learning_rate: 2.2420e-04

KeyboardInterrupt: 